In [20]:
import itertools
import json
from pathlib import Path

import pandas as pd

In [21]:
def sorted_tuple(x):
    return tuple(sorted(x))

In [22]:
from adapt.dvc import load_experiments

filepaths = list(Path("../../tmp/erx/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[0]

42 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'task': 'erx',
  'train': {'dataset': {'path': 'bdsaglam/web_nlg-erx-concat',
    'name': 'release_v3.0_en',
    'split': 'train[:100]'},
   'optimizer': 'noop',
   'ensemble': 'no'},
  'evaluation': {'dataset': {'path': 'bdsaglam/web_nlg-erx-concat',
    'name': 'release_v3.0_en',
    'split': 'dev[:1000]'}},
  'program': {'prompting': 'structured'},
  'lm': {'model': 'llama-3-8b', 'temperature': 0.0},
  'run': 1},
 'metrics': {'exact.precision': 0.014585968840764236,
  'exact.recall': 0.013411106866989222,
  'exact.f1': 0.013893968089212146,
  'fuzzy.precision': 0.2980273675422333,
  'fuzzy.recall': 0.2736804602675239,
  'fuzzy.f1': 0.28203483319061007}}

In [23]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
print(f"{len(df)} experiments before preprocessing")
df


42 experiments before preprocessing


,id,name,params.task,params.train.dataset.path,params.train.dataset.name,params.train.dataset.split,params.train.optimizer,params.train.ensemble,params.evaluation.dataset.path,params.evaluation.dataset.name,...,params.program.prompting,params.lm.model,params.lm.temperature,params.run,metrics.exact.precision,metrics.exact.recall,metrics.exact.f1,metrics.fuzzy.precision,metrics.fuzzy.recall,metrics.fuzzy.f1
0,workspace,None,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.014586,0.013411,0.013894,0.298027,0.273680,0.282035
1,8a95fd4dc54ea708f0f7421b291c9ec0e1c8f550,lathy-jaws,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,sft,llama-3-8b-sft,0.0,low,0.630100,0.622543,0.623483,0.850379,0.847531,0.844171
2,10018c1a534cbb1bb093856fe741782b063885d5,bluff-dita,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],bfsrs-high,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.280462,0.275396,0.276618,0.676080,0.664274,0.666510
3,d3bb54eb8efdc0a2bb5c86befb6b9c5cd0159ebf,irony-gust,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],miprov2-medium,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.250988,0.233233,0.240662,0.653248,0.607018,0.625459
4,38621dd5eb7f657d0ed4c3060ec1bfdaab830cda,outer-ring,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],bfsrs-medium,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.255153,0.246698,0.249767,0.651775,0.629756,0.637432
5,207289e5a394e3f9ae82cf08aea3d2ca07e190ba,moral-prof,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,sft,llama-3-8b-sft,0.0,high,0.945660,0.939192,0.941871,0.980038,0.973275,0.976060
6,e20f45a313f10b09f83c05fdb44f275d52da7799,spicy-teff,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,sft,llama-3-8b-sft,0.0,medium,0.826799,0.806647,0.814956,0.942006,0.920719,0.929080
7,8af6e338f987cfa912d7ee566a61bd24f956fd7c,epoxy-dele,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],miprov2-light,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.164665,0.159790,0.161087,0.549267,0.538236,0.540178
8,c4c203b34fff9acc3052b01148457799d70463a8,girly-cuss,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.018212,0.017030,0.017500,0.356735,0.335013,0.341740
9,workspace,None,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.014586,0.013411,0.013894,0.298027,0.273680,0.282035


In [24]:
param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]

In [25]:
if 'params.program.prompting' not in df.columns:
    df['params.program.prompting'] = 'structured'
df['params.program.prompting'] = df['params.program.prompting'].fillna('structured')

In [26]:
df.dropna(subset=param_cols + metric_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")
df.head()

24 experiments after preprocessing


,id,name,params.task,params.train.dataset.path,params.train.dataset.name,params.train.dataset.split,params.train.optimizer,params.train.ensemble,params.evaluation.dataset.path,params.evaluation.dataset.name,...,params.program.prompting,params.lm.model,params.lm.temperature,params.run,metrics.exact.precision,metrics.exact.recall,metrics.exact.f1,metrics.fuzzy.precision,metrics.fuzzy.recall,metrics.fuzzy.f1
0,workspace,None,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.014586,0.013411,0.013894,0.298027,0.273680,0.282035
1,8a95fd4dc54ea708f0f7421b291c9ec0e1c8f550,lathy-jaws,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],noop,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,sft,llama-3-8b-sft,0.0,low,0.630100,0.622543,0.623483,0.850379,0.847531,0.844171
2,10018c1a534cbb1bb093856fe741782b063885d5,bluff-dita,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],bfsrs-high,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.280462,0.275396,0.276618,0.676080,0.664274,0.666510
3,d3bb54eb8efdc0a2bb5c86befb6b9c5cd0159ebf,irony-gust,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],miprov2-medium,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.250988,0.233233,0.240662,0.653248,0.607018,0.625459
4,38621dd5eb7f657d0ed4c3060ec1bfdaab830cda,outer-ring,erx,bdsaglam/web_nlg-erx-concat,release_v3.0_en,train[:100],bfsrs-medium,no,bdsaglam/web_nlg-erx-concat,release_v3.0_en,...,structured,llama-3-8b,0.0,1,0.255153,0.246698,0.249767,0.651775,0.629756,0.637432


In [27]:
for col in param_cols:
    values = list(df[col].unique())
    print(f"- {col}: {values}")
    print()

- params.task: ['erx']

- params.train.dataset.path: ['bdsaglam/web_nlg-erx-concat']

- params.train.dataset.name: ['release_v3.0_en']

- params.train.dataset.split: ['train[:100]']

- params.train.optimizer: ['noop', 'bfsrs-high', 'miprov2-medium', 'bfsrs-medium', 'miprov2-light']

- params.train.ensemble: ['no']

- params.evaluation.dataset.path: ['bdsaglam/web_nlg-erx-concat']

- params.evaluation.dataset.name: ['release_v3.0_en']

- params.evaluation.dataset.split: ['dev[:1000]', 'dev', 'dev[:100]']

- params.program.prompting: ['structured', 'sft']

- params.lm.model: ['llama-3-8b', 'llama-3-8b-sft', 'qwen-2.5-32b']

- params.lm.temperature: [np.float64(0.0), np.float64(0.5)]

- params.run: [1, 'low', 'high', 'medium']



In [28]:
df.to_json('exps.jsonl', orient='records', lines=True)

## Setup remaining experiments

In [29]:
def produce_experiment_configs(common_params, varying_params):
    # Generate all possible combinations of parameters
    varying_params = {**common_params, **varying_params}
    keys = varying_params.keys()
    values = varying_params.values()
    for instance in itertools.product(*values):
        yield dict(zip(keys, instance))

In [30]:
def produce_all_experiment_configs(common_params: dict, varying_params_list: list[dict]):
    for params in varying_params_list:
        for exp_config in produce_experiment_configs(common_params, params):
            yield exp_config

In [31]:
common_params = {
    "params.task": ["erx"],
    "params.train.dataset.path": ["bdsaglam/web_nlg-erx-concat"],
    "params.train.dataset.name": ["release_v3.0_en"],
    "params.train.dataset.split": ['"train[:100]"'],
    "params.evaluation.dataset.path": ["bdsaglam/web_nlg-erx-concat"],
    "params.evaluation.dataset.name": ["release_v3.0_en"],
    "params.evaluation.dataset.split": ['"dev"'],
    "params.train.ensemble": [
        "no",
        # "yes",
    ],
    "params.lm.temperature": [
        0.0,
        # 0.5,
        # 0.7,
    ],
    "params.run": [
        1,
        # 2,
        # 3,
    ],
}

In [32]:
varying_params_list = [
    {
        "params.train.optimizer": [
            "noop",
            "bfsrs-medium",
            "bfsrs-high",
            'bfsrs-ulti',
            "miprov2-light",
            "miprov2-medium",
        ],
        "params.program.prompting": ["structured"],
        "params.lm.model": [
            "llama-3-8b",
            # "qwen-2.5-32b",
        ],
    },
    # {
    #     "params.train.optimizer": ["noop"],
    #     "params.program.prompting": ["sft"],
    #     "params.lm.model": [
    #         "llama-3-8b-sft",
    #     ],
    # },
]

In [33]:
exp_configs = list(produce_all_experiment_configs(common_params, varying_params_list))
target_params = list(exp_configs[0].keys())
print(f"{len(exp_configs)} experiment configurations")
print(target_params)

6 experiment configurations
['params.task', 'params.train.dataset.path', 'params.train.dataset.name', 'params.train.dataset.split', 'params.evaluation.dataset.path', 'params.evaluation.dataset.name', 'params.evaluation.dataset.split', 'params.train.ensemble', 'params.lm.temperature', 'params.run', 'params.train.optimizer', 'params.program.prompting', 'params.lm.model']


In [34]:
if len(df):
    existing_configs = df[target_params].to_dict(orient="records")
    existing_configs[0]
else:
    existing_configs = []

print("Existing exps:", len(existing_configs))

Existing exps: 24


In [35]:
# find the missing configurations
missing_configs = [
    dict(kv)
    for kv in list(
        {tuple(sorted(config.items())) for config in exp_configs}
        - {tuple(sorted(config.items())) for config in existing_configs}
    )
]
print(f"{len(missing_configs)} missing configurations")

6 missing configurations


In [36]:
def make_command(exp_config):
    lines = ["dvc exp run --queue"]
    for target_param in target_params:
        arg_name = target_param.split(".", 1)[-1]
        arg_value = exp_config[target_param]
        lines.append(f"-S {arg_name}='{arg_value}'")

    command = " \\\n    ".join(lines)
    return command

In [37]:
with open("run.sh", "w") as f:
    f.write("#!/bin/sh\n\n")
    for exp_config in missing_configs:
        f.write(make_command(exp_config))
        f.write("\n\n")

## Inspect

In [38]:
df[['name', 'params.program.prompting', 'params.run', 'params.train.optimizer', 'params.lm.model', 'params.lm.temperature', *metric_cols]].sort_values(by='metrics.fuzzy.f1', ascending=False)

,name,params.program.prompting,params.run,params.train.optimizer,params.lm.model,params.lm.temperature,metrics.exact.precision,metrics.exact.recall,metrics.exact.f1,metrics.fuzzy.precision,metrics.fuzzy.recall,metrics.fuzzy.f1
5,moral-prof,sft,high,noop,llama-3-8b-sft,0.0,0.945660,0.939192,0.941871,0.980038,0.973275,0.976060
6,spicy-teff,sft,medium,noop,llama-3-8b-sft,0.0,0.826799,0.806647,0.814956,0.942006,0.920719,0.929080
1,lathy-jaws,sft,low,noop,llama-3-8b-sft,0.0,0.630100,0.622543,0.623483,0.850379,0.847531,0.844171
37,alone-mesh,structured,1,bfsrs-medium,qwen-2.5-32b,0.0,0.366102,0.358353,0.361248,0.793954,0.783383,0.786429
19,metal-pons,structured,1,bfsrs-medium,qwen-2.5-32b,0.0,0.377839,0.367084,0.370645,0.797541,0.779191,0.783746
20,crumb-geum,sft,1,noop,llama-3-8b,0.0,0.451251,0.434675,0.437976,0.762150,0.740962,0.742936
38,slack-poss,structured,1,bfsrs-medium,llama-3-8b,0.0,0.270941,0.263430,0.266274,0.686891,0.661890,0.671487
2,bluff-dita,structured,1,bfsrs-high,llama-3-8b,0.0,0.280462,0.275396,0.276618,0.676080,0.664274,0.666510
28,modal-huck,structured,1,bfsrs-high,llama-3-8b,0.5,0.274224,0.268814,0.269906,0.666549,0.656943,0.657637
4,outer-ring,structured,1,bfsrs-medium,llama-3-8b,0.0,0.255153,0.246698,0.249767,0.651775,0.629756,0.637432
